In [ ]:
import os
import bs4
import json
import time
import lxml
import deepl
import requests
import cloudscraper
from pathlib import Path
from copy import deepcopy
from googletrans import Translator
from difflib import SequenceMatcher
from fake_useragent import UserAgent
from IPython.display import clear_output as clear

In [ ]:
ua = UserAgent()
translator = Translator()
dl_translator = deepl.Translator("4a2f6ee1-0d7a-8c77-9b1d-9476a086f204:fx")

In [ ]:
path = os.path.abspath('').replace('src', '')
top_dir = Path(path).parent.absolute()
conjugations_data_dir = os.path.join(top_dir, 'conjugations', 'data', 'language-specific')
out_dir = os.path.join(path, 'out')

In [ ]:
headers = {"accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
           "accept-encoding": "gzip, deflate, br",
           "accept-language": "en-GB,en;q=0.9,es-ES;q=0.8,es;q=0.7,en-US;q=0.6,eu;q=0.5",
           "cache-control": "max-age=0",
           "cookie": "t=238707487; _ga=GA1.2.1376835774.1641262578; _gid=GA1.2.1482423077.1641262578; _fbp=fb.1.1641262579526.851471446",
           "referer": "https://hidemy.name/en/proxy-list/?start=64",
           "sec-ch-ua-mobile": "?0",
           "sec-ch-ua-platform": "macOS",
           "sec-fetch-dest": "document",
           "sec-fetch-mode": "navigate",
           "sec-fetch-site": "same-origin",
           "sec-fetch-user": "?1",
           "upgrade-insecure-requests": "1",
           "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"}

In [ ]:
language = 'spanish'

In [ ]:
with open(os.path.join(out_dir, 'graded', f'translations_{language}.json'), 'r', encoding = 'utf8') as file:
    translations = json.loads(file.read())

with open(os.path.join(conjugations_data_dir, f'conjugations_{language}.json'), 'r', encoding = 'utf8') as file:
    conjugations = json.loads(file.read())

In [ ]:
output = { verb: {} for verb in translations }

In [ ]:
if language == "italian":
    translations.pop( "ridefinire" )

In [ ]:
modal_verbs = ['can', 'could', 'may', 'might', 'must', 'shall', 'should', 'will', 'would']
translations = {verb: {'metadata': translations[verb]['metadata'], 'weighted': translations[verb]['weighted']} for verb in translations}

In [ ]:
for verb in translations:
    if translations[verb]['metadata']['principal'] not in translations[verb]['weighted']:
        if translations[verb]['metadata']['principal'] + ' to' in translations[verb]['weighted']:
            translations[verb]['metadata']['principal'] += ' to'
    
    if translations[verb]['metadata']['consensus'] not in translations[verb]['weighted']:
        if translations[verb]['metadata']['consensus'] + ' to' in translations[verb]['weighted']:
            translations[verb]['metadata']['consensus'] += ' to'
    
    if translations[verb]['metadata']['principal'] not in translations[verb]['weighted']:
        if translations[verb]['metadata']['principal'][-3:] == ' to' and translations[verb]['metadata']['principal'][:-3] in translations[verb]['weighted']:
            translations[verb]['metadata']['principal'] = translations[verb]['metadata']['principal'][:-3] 
    
    if translations[verb]['metadata']['consensus'] not in translations[verb]['weighted']:
        if translations[verb]['metadata']['consensus'][-3:] == ' to' and translations[verb]['metadata']['consensus'][:-3] in translations[verb]['weighted']:
            translations[verb]['metadata']['consensus'] = translations[verb]['metadata']['consensus'][:-3] 

In [ ]:
def normalize(infinitive):    
    principal = translations[infinitive]['metadata']['principal']
    check = { 'unique': [], 'duplicates': [] }
    similar = { 'retain': [], 'discard': [] }

    heaviest_verb = list(translations[infinitive]['weighted'].keys())[0]
    heaviest_weight = translations[infinitive]['weighted'][heaviest_verb]
        
    if principal != heaviest_verb:
        translations[infinitive]['weighted'][principal] = heaviest_weight + 1
        heaviest_weight += 1

    formatted = deepcopy(translations[infinitive]['weighted'])

    formatted = dict(sorted(formatted.items(), key = lambda item: item[1])[::-1])

    if principal != list(formatted.keys())[0]:
        formatted = {**{principal: formatted[principal]}, **{translation: formatted[translation] for translation in formatted if translation != principal}}

    for candidate in formatted:
        for existing in check['unique']:
            if candidate in existing or existing in candidate:
                check['duplicates'].append(candidate)
                break
        if candidate not in check['duplicates']:
            check['unique'].append(candidate)

    for duplicate in check['duplicates']:
        formatted.pop(duplicate)

    for c1 in formatted:
        if c1 not in similar['discard']:
            for c2 in formatted:
                if c1 != c2 and c2 not in similar['retain'] and c2 not in similar['discard']:
                    if SequenceMatcher(None, c1, c2).ratio() > 0.75:
                        formatted[c1] += formatted[c2]

                        if formatted[c1] > heaviest_weight:
                            principal = c1
                            heaviest_weight = formatted[c1]

                        similar['discard'].append(c2)

    for similar in similar['discard']:
        formatted.pop(similar)

    formatted = dict(sorted(formatted.items(), key = lambda item: item[1])[::-1])

    output[infinitive]['principal'] = principal
    output[infinitive]['weighted'] = { translation: round(formatted[translation] / heaviest_weight, 4)  for translation in formatted }

In [ ]:
def getCollinsTranslations(verb, language):

    translations = []

    try:
        url = f'https://www.collinsdictionary.com/dictionary/{language}-english/{verb}'
        scraper = cloudscraper.create_scraper(
            browser={
                'browser': 'firefox',
                'platform': 'windows',
                'mobile': False
            })
        soup = bs4.BeautifulSoup(scraper.get(url, headers = {"useragent": f"{ua.random}"}).text, 'lxml')

        if 'Cloudflare' not in soup.select('title')[0].getText():

            for translation in soup.select('.page .dictionary .sense:not(.type-example)>.cit.type-translation .quote'):
                if translation.getText().strip()[:3] == 'to ':
                    translations.append(translation.getText().strip())

    except:
        pass


    return translations

In [ ]:
def getWiktionaryTranslations(verb, language):
    translations = []

    page = requests.get(f'https://en.wiktionary.org/wiki/{verb}#{language.capitalize()}')
    section = page.text.split(f'<h2><span class="mw-headline" id="{language.capitalize()}">')[1].split('<h2>')[0]
    soup = bs4.BeautifulSoup(section, 'lxml')

    for ol in soup.select('ol'):
        for li in ol.select('li'):
            translation = li.getText().split(') ')[1].split(' (')[0].split('\n')[0]
            translations.append(translation)

    return translations

In [ ]:
def getWordReferenceTranslation(verb, language):
    translations = []

    lang = {'spanish': 'spen', 'french': 'fren', 'german': 'deen', 'italian': 'iten', 'portuguese': 'pten'}
    page = requests.get(f'https://www.wordreference.com/{lang[language]}/{verb}')
    soup = bs4.BeautifulSoup(page.text, 'lxml')

    for section in soup.select('.WRD'):

        append = False

        for row in section.find_all("tr", class_ = ['even','odd']):
            
            if row.select('.ToWrd'):
            
                if row.select('.FrWrd'):
                    origin = row.select('.FrWrd')[0]
                    
                    if origin.select('strong')[0].getText() == verb and origin.select('em')[0].getText().lower()[0] == 'v':
                        append = True
                    else:
                        append = False
                                
                if append and row.select('.ToWrd'):
                    raw = row.select('.ToWrd')[0].getText()

                    if row.select('.ToWrd')[0].select('em')[0].getText()[0] == 'v':
                        translation = raw.split('⇒')[0].split(' v ')[0].split(' vi ')[0].split(' vtr ')[0]
                        translations.append(translation)

    return translations

In [ ]:
modals = []
modals_principal = []

for verb in translations:
    output[verb] = {}
    
    if translations[verb]['metadata']['principal'] in modal_verbs:
        modals_principal.append( (verb, conjugations[verb]['rank']) )
    for translation in translations[verb]['weighted']:
        if translation in modal_verbs:
            modals.append( (verb, conjugations[verb]['rank']) )
            break

In [ ]:
for verb in modals:
    infinitive,rank = verb

    for modal in modal_verbs:
        if modal in translations[infinitive]['weighted']:
            translations[infinitive]['weighted'].pop(modal)

for verb in modals_principal:
    infinitive,rank = verb

    translations[infinitive]['metadata']['principal'] = translations[infinitive]['metadata']['consensus']

for verb in modals:
    normalize(verb)

# print(output)

In [ ]:
disagree = [(verb, conjugations[verb]['rank']) for verb in translations if translations[verb]['metadata']['principal'] != translations[verb]['metadata']['consensus']]

In [ ]:
context = {}

for i in range(0, len(disagree)):

    verb,rank = disagree[i]

    context[verb] = {}

    context[verb]['selected'] = ''

    context[verb]['principal'] = translations[verb]['metadata']['principal']
    context[verb]['primaries'] = translations[verb]['metadata']['primaries']

    try: context[verb]['collins'] = getCollinsTranslations(verb, language)[:5]
    except: pass
    
    try: context[verb]['wiktionary'] = getWiktionaryTranslations(verb, language)[:5]
    except: pass

    try: context[verb]['wordreference'] = getWordReferenceTranslation(verb, language)[:5]
    except: pass

    clear(); print(f'{verb} ({round(((i + 1)/len(disagree)) * 100, 3)}%)')

In [ ]:
with open(os.path.join(path, 'corrections', 'details', f'corrections_{language}.json'), 'w', encoding = 'utf8') as file:
    json.dump(context, file, indent = 4, ensure_ascii = False)